掘金，keras源码分析
https://juejin.im/search?query=keras%E6%BA%90%E7%A0%81%E5%88%86%E6%9E%90


In [1]:
from __future__ import print_function
import keras
import keras.backend as K
from keras.datasets import cifar100
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os


Using TensorFlow backend.


In [2]:
# 设置
batch_size = 512
num_classes = 100
epochs = 50
data_augmentation = False
num_predictions = 20


In [3]:
# 导入数据
# (x_train, y_train), (x_test, y_test) = cifar100.load_data()
from six.moves import cPickle
def load_cifar100_my(path):
    with open(os.path.join(os.getcwd(),path), 'rb') as f:
        d = cPickle.load(f, encoding='bytes')
        d_decoded = {}
        for k, v in d.items():
            d_decoded[k.decode('utf8')] = v
            d = d_decoded
        data = d['data']
        labels = d['fine_labels']
        data = data.reshape(data.shape[0], 3, 32, 32)
        data = data.transpose(0,2,3,1)
    return data, labels
x_train, y_train = load_cifar100_my(r'data\cifar-100-python\train')
x_test , y_test  = load_cifar100_my(r'data\cifar-100-python\test')
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# 数据预处理
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# x_train = x_train.astype('float16')
# x_test  = x_test.astype('float16')
# y_train = y_train.astype('float16')
# y_test  = y_test.astype('float16')


x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [4]:
print(y_train.shape)
print(y_train.dtype)
x_train.shape[1:]

(50000, 100)
float32


In [5]:
# 查看数据
import matplotlib.pyplot as plt
plt.figure(figsize=(1,1))
plt.imshow(x_train[101])
plt.axis('off')
plt.show()

<Figure size 100x100 with 1 Axes>

In [7]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', activation='relu',
                 input_shape=x_train.shape[1:]))

model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes))
model.add(Activation('softmax'))


In [92]:
import tensorflow as tf

In [88]:
# Custom loss and metrics
# 如果是多分类多标签呢？
def mean_pred(y_true, y_pred):
    return K.mean(y_pred)

def margin_loss(y_true, y_pred):
    lamb, margin = 0.5, 0.1
    return K.sum(y_true * K.square(K.relu(1 - margin - y_pred)) + lamb * (
        1 - y_true) * K.square(K.relu(y_pred - margin)), axis=-1)

In [86]:
# loss, optimizer, metrics

loss       = keras.losses.categorical_crossentropy
# loss       = keras.losses.mean_squared_error

# optimizer  = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
optimizer  = keras.optimizers.sgd(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)

metrics    = [keras.metrics.categorical_accuracy]
# metrics    = [keras.metrics.categorical_accuracy, keras.metrics.mae]

model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

In [91]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)

Not using data augmentation.
Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 7s 149us/step - loss: 3.7287 - categorical_accuracy: 0.1289 - val_loss: 3.5941 - val_categorical_accuracy: 0.1741
Epoch 2/50
50000/50000 [==============================] - 7s 141us/step - loss: 3.7204 - categorical_accuracy: 0.1325 - val_loss: 3.5842 - val_categorical_accuracy: 0.1735
Epoch 3/50
50000/50000 [==============================] - 7s 141us/step - loss: 3.7066 - categorical_accuracy: 0.1360 - val_loss: 3.5725 - val_categorical_accuracy: 0.1739
Epoch 4/50
50000/50000 [==============================] - 7s 142us/step - loss: 3.6990 - categorical_accuracy: 0.1352 - val_loss: 3.5666 - val_categorical_accuracy: 0.1754
Epoch 5/50
50000/50000 [==============================] - 7s 141us/step - loss: 3.6879 - categorical_accuracy: 0.1363 - val_loss: 3.5514 - val_categorical_accuracy: 0.1763
Epoch 6/50
50000/50000 [==============================] - 7s 

In [90]:
# 训练效果
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 1s 145us/step
Test loss: 3.601405978393555
Test accuracy: 0.1707


In [80]:
# 模型保存路径
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar100_trained_model.h5'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)

In [12]:
# 保存模型
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at d:\sync-cs\bluoveGitHub\note-on-ai\keras-helloworld\saved_models\keras_cifar100_trained_model.h5 


In [89]:
# 读取模型
# model = keras.models.load_model(model_path)

In [61]:
model.fit?

Signature: model.fit(x=None, y=None, batch_size=None, epochs=1, verbose=1, callbacks=None, validation_split=0.0, validation_data=None, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None, **kwargs)
Docstring:
Trains the model for a given number of epochs (iterations on a dataset).

# Arguments
    x: Numpy array of training data (if the model has a single input),
        or list of Numpy arrays (if the model has multiple inputs).
        If input layers in the model are named, you can also pass a
        dictionary mapping input names to Numpy arrays.
        `x` can be `None` (default) if feeding from
        framework-native tensors (e.g. TensorFlow data tensors).
    y: Numpy array of target (label) data
        (if the model has a single output),
        or list of Numpy arrays (if the model has multiple outputs).
        If output layers in the model are named, you can also pass a
        dictionary mapping output names

In [54]:
[i.shape for i in model.get_weights()]

(3, 3, 3, 32)
(32,)
(3, 3, 32, 32)
(32,)
(3, 3, 32, 64)
(64,)
(3, 3, 64, 64)
(64,)
(4096, 512)
(512,)
(512, 100)
(100,)


In [53]:
print(model.inputs)
model.summary()

[<tf.Tensor 'conv2d_1_input:0' shape=(?, 32, 32, 3) dtype=float32>]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 64)        36928     
_________________________________________________________________
max_pool